In [ ]:
from inait_forecasting_client import (
    predict,
    backtest,
    plot,
    read_file,
    compare,
    compare_plot,
)

### Import dataset and define config 

In [ ]:
data = read_file("../data/FMSA_Articles.csv", index_col=0)

config_common = {
    "data": data,
    "target_columns": "Article submissions",
    "forecasting_horizon": 6,
    "model": "inait-best",
}
config_predict = config_common.copy()

config_backtest = config_common.copy()
config_backtest.update(
    {
        "backtest_size": 12 * 2,  # last 2 years for backtest
        # "prediction_stride": 6,  # make prediction every 6 months
    }
)

### 1. Predict

In [ ]:
predict_results = predict(**config_predict)
plot(
    historical_data=data.loc[:, config_predict["target_columns"]],
    predicted_data=predict_results["prediction"],
)

### 2. Backtest

In [ ]:
backtest_results = backtest(**config_backtest)
plot(
    historical_data=data.loc[:, config_backtest["target_columns"]],
    predicted_data=backtest_results["prediction"],
)

In [ ]:
comparison_session_ids = {
    "inait-best": backtest_results["session_id"],
    "ridge": None,
    "mean_10": None,
    "naive": None,
}  # cached session ids for demo

In [ ]:
compare_results = compare(
    models=comparison_session_ids.keys(),
    session_ids=comparison_session_ids,
    **config_backtest,
)
compare_plot(compare_results["scores_df"], plot_type="bar")

### Extra: Predict using optimal model based on backtest result

In [ ]:
if compare_results["optimal_model"] == "inait-best":
    print("The optimal model is the same as the backtested model.")
    config_optimal = config_predict.copy()
else:
    print(f"Running predictions with optimal model: {compare_results['optimal_model']}")
    config_optimal = config_common.copy()
    config_optimal["model"] = compare_results["optimal_model"]
if compare_results["optimal_model"] == "naive" or compare_results[
    "optimal_model"
].startswith("mean"):
    raise ValueError("Naive and Mean baselines are not yet supported for predict.")
else:
    optimal_predict_results = predict(**config_optimal)
    plot(
        historical_data=data.loc[:, config_optimal["target_columns"]],
        predicted_data=optimal_predict_results["prediction"],
    )